In [1]:
import pandas as pd

In [4]:
import requests
import json
from typing import Optional, Dict, List, Any
import pandas as pd


class ZentereAPIClient:
    """
    Dynamic Python client for Zentere API with full CRUD operations.
    Optimized for Jupyter Notebook usage with pandas integration.
    """
    
    def __init__(self, base_url: str = "https://app-api-dev.zentere.com/api/v2"):
        self.base_url = base_url
        self.access_token = None
        self.token_type = "Bearer"
        
        # Default credentials
        self.client_id = "kLPrcbXlsHYelbpm5HzKg8ZgDE2rVXRhGyJ0GdqH"
        self.client_secret = "IbqUkvq1hWTuc6jK7X6xGClTLThshJhfU6nf7uYm"
    
    def authenticate(self, username: str, password: str) -> Dict[str, Any]:
        """
        Authenticate and get access token.
        
        Args:
            username: User's username
            password: User's password
            
        Returns:
            Dictionary containing token information
        """
        url = f"{self.base_url}/authentication/oauth2/token"
        
        data = {
            'grant_type': 'password',
            'client_id': self.client_id,
            'username': username,
            'password': password,
            'client_secret': self.client_secret
        }
        
        try:
            response = requests.post(url, data=data, timeout=30)
            response.raise_for_status()
            
            token_data = response.json()
            self.access_token = token_data.get('access_token')
            self.token_type = token_data.get('token_type', 'Bearer')
            
            print("=" * 60)
            print("✓ AUTHENTICATION SUCCESSFUL!")
            print("=" * 60)
            print(f"Token Type: {self.token_type}")
            print(f"Expires In: {token_data.get('expires_in')} seconds")
            print(f"Scope: {token_data.get('scope', 'N/A')}")
            print("=" * 60)
            
            return token_data
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Authentication failed: {str(e)}")
            if hasattr(e.response, 'text'):
                print(f"Response: {e.response.text}")
            raise
    
    def _get_headers(self) -> Dict[str, str]:
        """Get authorization headers."""
        if not self.access_token:
            raise Exception("❌ Not authenticated. Call authenticate() first.")
        
        return {
            'Authorization': f'{self.token_type} {self.access_token}',
            'Content-Type': 'application/json'
        }
    
    def search_read(self, 
                    model: str,
                    fields: Optional[List[str]] = None,
                    domain: Optional[List] = None,
                    limit: int = 100,
                    offset: int = 0,
                    order: Optional[str] = None,
                    as_dataframe: bool = False) -> Any:
        """
        Search and read records from a model.
        
        Args:
            model: Model name (e.g., 'data_feeds', 'forecast_model')
            fields: List of fields to retrieve (e.g., ['id', 'name', 'date'])
            domain: Search domain filter (e.g., [['active', '=', True]])
            limit: Maximum number of records to return
            offset: Number of records to skip
            order: Sort order (e.g., 'name asc', 'date desc')
            as_dataframe: Return as pandas DataFrame instead of list
            
        Returns:
            List of records as dictionaries or pandas DataFrame
        """
        url = f"{self.base_url}/search_read"
        
        params = {'model': model}
        
        if fields:
            params['fields'] = json.dumps(fields)
        if domain:
            params['domain'] = json.dumps(domain)
        if limit:
            params['limit'] = limit
        if offset:
            params['offset'] = offset
        if order:
            params['order'] = order
        
        try:
            response = requests.post(url, headers=self._get_headers(), params=params, timeout=30)
            response.raise_for_status()
            
            data = response.json()
            
            print(f"✓ Retrieved {len(data)} records from '{model}'")
            
            if as_dataframe and data:
                return pd.DataFrame(data)
            return data
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Search failed: {str(e)}")
            if hasattr(e.response, 'text'):
                print(f"Response: {e.response.text}")
            raise
    
    def create(self, model: str, values: Dict[str, Any]) -> int:
        """
        Create a new record.
        
        Args:
            model: Model name
            values: Dictionary of field values
            
        Returns:
            ID of the created record
        """
        url = f"{self.base_url}/create"
        params = {'model': model}
        
        try:
            response = requests.post(
                url,
                headers=self._get_headers(),
                params=params,
                json=values,
                timeout=30
            )
            response.raise_for_status()
            
            result = response.json()
            record_id = result if isinstance(result, int) else result.get('id')
            print(f"✓ Created record with ID: {record_id} in '{model}'")
            return record_id
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Create failed: {str(e)}")
            if hasattr(e.response, 'text'):
                print(f"Response: {e.response.text}")
            raise
    
    def write(self, 
              model: str, 
              record_id: int, 
              values: Dict[str, Any]) -> bool:
        """
        Update an existing record.
        
        Args:
            model: Model name
            record_id: ID of the record to update
            values: Dictionary of field values to update
            
        Returns:
            True if successful
        """
        url = f"{self.base_url}/write"
        params = {'model': model}
        
        payload = {
            'id': record_id,
            'values': values
        }
        
        try:
            response = requests.put(
                url,
                headers=self._get_headers(),
                params=params,
                json=payload,
                timeout=30
            )
            response.raise_for_status()
            
            print(f"✓ Updated record ID: {record_id} in '{model}'")
            return True
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Update failed: {str(e)}")
            if hasattr(e.response, 'text'):
                print(f"Response: {e.response.text}")
            raise
    
    def unlink(self, model: str, record_id: int) -> bool:
        """
        Delete a record.
        
        Args:
            model: Model name
            record_id: ID of the record to delete
            
        Returns:
            True if successful
        """
        url = f"{self.base_url}/unlink"
        params = {'model': model}
        
        payload = {'id': record_id}
        
        try:
            response = requests.delete(
                url,
                headers=self._get_headers(),
                params=params,
                json=payload,
                timeout=30
            )
            response.raise_for_status()
            
            print(f"✓ Deleted record ID: {record_id} from '{model}'")
            return True
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Delete failed: {str(e)}")
            if hasattr(e.response, 'text'):
                print(f"Response: {e.response.text}")
            raise
    
    def search_count(self, model: str, domain: Optional[List] = None) -> int:
        """
        Count records matching a domain.
        
        Args:
            model: Model name
            domain: Search domain filter
            
        Returns:
            Count of matching records
        """
        url = f"{self.base_url}/search_count"
        params = {'model': model}
        
        if domain:
            params['domain'] = json.dumps(domain)
        
        try:
            response = requests.post(url, headers=self._get_headers(), params=params, timeout=30)
            response.raise_for_status()
            
            count = response.json()
            print(f"✓ Found {count} records in '{model}'")
            return count
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Count failed: {str(e)}")
            if hasattr(e.response, 'text'):
                print(f"Response: {e.response.text}")
            raise


# ============================================================================
# JUPYTER NOTEBOOK QUICK START
# ============================================================================

print("""
╔════════════════════════════════════════════════════════════════════════╗
║                  ZENTERE API CLIENT - QUICK START                      ║
╚════════════════════════════════════════════════════════════════════════╝

STEP 1: Initialize client
    client = ZentereAPIClient()

STEP 2: Authenticate
    client.authenticate('martin@demo.com', 'demo')

STEP 3: Read data
    # As list of dictionaries
    data = client.search_read('data_feeds', limit=10)
    
    # As pandas DataFrame
    df = client.search_read('data_feeds', limit=10, as_dataframe=True)
    
    # With specific fields
    data = client.search_read('data_feeds', 
                             fields=['id', 'name', 'create_date'],
                             limit=10)
    
    # With filters
    data = client.search_read('forecast_model',
                             domain=[['date', '>=', '2024-01-01']],
                             limit=50)

Ready to test! Copy the commands above to your notebook.
""")


# ============================================================================
# TEST SCRIPT - Uncomment to run automated tests
# ============================================================================

def run_validation_tests():
    """
    Run validation tests - Use this in your notebook to test the API
    """
    print("\n" + "="*70)
    print("STARTING VALIDATION TESTS")
    print("="*70)
    
    # Test 1: Initialize
    print("\n[TEST 1] Initializing client...")
    client = ZentereAPIClient()
    print("✓ Client initialized")
    
    # Test 2: Authenticate
    print("\n[TEST 2] Authenticating...")
    try:
        token_info = client.authenticate('martin@demo.com', 'demo')
        print("✓ Authentication test PASSED")
    except Exception as e:
        print(f"✗ Authentication test FAILED: {e}")
        return
    
    # Test 3: Read data_feeds
    print("\n[TEST 3] Reading data_feeds table...")
    try:
        data = client.search_read('data_feeds', limit=5)
        print(f"✓ Read test PASSED - Got {len(data)} records")
        if data:
            print(f"\nSample record keys: {list(data[0].keys())}")
            print(f"First record: {json.dumps(data[0], indent=2, default=str)}")
    except Exception as e:
        print(f"✗ Read test FAILED: {e}")
    
    # Test 4: Read with specific fields
    print("\n[TEST 4] Reading with specific fields...")
    try:
        data = client.search_read('data_feeds', 
                                 fields=['id', 'name'],
                                 limit=3)
        print(f"✓ Filtered fields test PASSED")
        print(f"Records: {json.dumps(data, indent=2, default=str)}")
    except Exception as e:
        print(f"✗ Filtered fields test FAILED: {e}")
    
    # Test 5: Count records
    print("\n[TEST 5] Counting records...")
    try:
        count = client.search_count('data_feeds')
        print(f"✓ Count test PASSED - Total records: {count}")
    except Exception as e:
        print(f"✗ Count test FAILED: {e}")
    
    # Test 6: Try pandas DataFrame
    print("\n[TEST 6] Converting to pandas DataFrame...")
    try:
        df = client.search_read('data_feeds', limit=5, as_dataframe=True)
        print(f"✓ DataFrame test PASSED")
        print(f"\nDataFrame shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        print(f"\nFirst 3 rows:")
        print(df.head(3))
    except Exception as e:
        print(f"✗ DataFrame test FAILED: {e}")
    
    # Test 7: Read forecast_model
    print("\n[TEST 7] Reading forecast_model table...")
    try:
        data = client.search_read('forecast_model', limit=5)
        print(f"✓ Forecast model test PASSED - Got {len(data)} records")
        if data:
            print(f"Sample record: {json.dumps(data[0], indent=2, default=str)}")
    except Exception as e:
        print(f"✗ Forecast model test FAILED: {e}")
    
    print("\n" + "="*70)
    print("VALIDATION TESTS COMPLETED")
    print("="*70)


# Uncomment the line below to run validation tests
# run_validation_tests()


╔════════════════════════════════════════════════════════════════════════╗
║                  ZENTERE API CLIENT - QUICK START                      ║
╚════════════════════════════════════════════════════════════════════════╝

STEP 1: Initialize client
    client = ZentereAPIClient()

STEP 2: Authenticate
    client.authenticate('martin@demo.com', 'demo')

STEP 3: Read data
    # As list of dictionaries
    data = client.search_read('data_feeds', limit=10)
    
    # As pandas DataFrame
    df = client.search_read('data_feeds', limit=10, as_dataframe=True)
    
    # With specific fields
    data = client.search_read('data_feeds', 
                             fields=['id', 'name', 'create_date'],
                             limit=10)
    
    # With filters
    data = client.search_read('forecast_model',
                             domain=[['date', '>=', '2024-01-01']],
                             limit=50)

Ready to test! Copy the commands above to your notebook.



In [5]:
run_validation_tests()


STARTING VALIDATION TESTS

[TEST 1] Initializing client...
✓ Client initialized

[TEST 2] Authenticating...
✓ AUTHENTICATION SUCCESSFUL!
Token Type: Bearer
Expires In: 3600 seconds
Scope: 
✓ Authentication test PASSED

[TEST 3] Reading data_feeds table...
✓ Retrieved 5 records from 'data_feeds'
✓ Read test PASSED - Got 5 records

Sample record keys: ['activity_calendar_event_id', 'activity_date_deadline', 'activity_exception_decoration', 'activity_exception_icon', 'activity_ids', 'activity_state', 'activity_summary', 'activity_type_icon', 'activity_type_id', 'activity_user_id', 'bpo_id', 'bpo_ids', 'business_unit_id', 'calendar_week_id', 'client_id', 'create_date', 'create_uid', 'data_type', 'date', 'description', 'display_name', 'forecast_value', 'has_message', 'id', 'lob_id', 'message_attachment_count', 'message_follower_ids', 'message_has_error', 'message_has_error_counter', 'message_has_sms_error', 'message_ids', 'message_is_follower', 'message_needaction', 'message_needaction_cou